In [1]:
!git clone https://github.com/nudojacopo/thesis.git

Cloning into 'thesis'...
remote: Enumerating objects: 1269, done.
remote: Counting objects: 100% (518/518), done.
remote: Compressing objects: 100% (288/288), done.
remote: Total 1269 (delta 357), reused 343 (delta 229), pack-reused 751
Receiving objects: 100% (1269/1269), 64.35 MiB | 31.57 MiB/s, done.
Resolving deltas: 100% (800/800), done.


In [78]:
# Import and directories
root='/kaggle/working/'
import sys
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from scipy import stats
import random
from scipy.stats import chi2
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot
!pip install powerlaw
import powerlaw
!pip install gdown
sys.path.append('/kaggle/working/thesis/src/PRO/PRO_package')
from functions import *

# Parameters to set up
platform='facebook_brexit'
format='csv'

# Link for data sources
link_for_drive={'facebook_brexit':'1KGH2s8sPnvXZc7_wVgTCoGTXPr3B7SCe'}

In [88]:
url='https://drive.google.com/uc?id='+link_for_drive[platform]
output='original_dataset.'+format
gdown.download(url,output,quiet=False)

if format=='csv':
    data=pd.read_csv('/kaggle/working/original_dataset.'+format)
if format=='parquet':
    data=pd.read_parquet('/kaggle/working/original_dataset.'+format)

Downloading...
From (original): https://drive.google.com/uc?id=1KGH2s8sPnvXZc7_wVgTCoGTXPr3B7SCe
From (redirected): https://drive.google.com/uc?id=1KGH2s8sPnvXZc7_wVgTCoGTXPr3B7SCe&confirm=t&uuid=cedd4acf-bb12-43e1-a6bd-72c9024ba62a
To: /kaggle/working/original_dataset.csv

  0%|          | 0.00/119M [00:00<?, ?B/s]
 10%|▉         | 11.5M/119M [00:00<00:00, 114MB/s]
 24%|██▍       | 28.8M/119M [00:00<00:00, 148MB/s]
 45%|████▍     | 53.5M/119M [00:00<00:00, 193MB/s]
 64%|██████▍   | 76.5M/119M [00:00<00:00, 206MB/s]
100%|██████████| 119M/119M [00:00<00:00, 196MB/s] 


In [89]:
print(data.columns)

# Mandatory to check 
post_id = "post_id"
toxicity_score='toxicity_score'
comment_id='comment_id'
user_id='from_id'
created_at='created_time'
topic='topic'
social='social'

# Often to add
'''data['topic']='Brexit'
data['social']='Facebook'
data['created_at']=data['created_time']
'''

Index(['Unnamed: 0', 'from_id', 'created_time', 'post_id', 'comment_id',
       'toxicity_score', 'message_y'],
      dtype='object')


"data['topic']='Brexit'\ndata['social']='Facebook'\ndata['created_at']=data['created_time']\n"

In [92]:
data['created_at']=data['created_time']
data['topic']='Brexit'
data['social']='Facebook'

In [93]:
comment_counts = data[post_id].value_counts()
threads_with_more_than_50_comments = comment_counts[comment_counts > 50].index
data = data[data[post_id].isin(threads_with_more_than_50_comments)]
threads = data.groupby(post_id)  # Divide in threads

progress_bar = tqdm(total=len(threads), desc="Add variables...")
for name, group in threads:
    group['time'] = pd.to_datetime(group[created_at],format='ISO8601')
    group.sort_values(by=created_at, inplace=True)
    group['thread_birth'] = group['time'].min()
    group['temporal_distance_birth_h'] = round(pd.to_timedelta(group['time']-group['thread_birth']).dt.total_seconds() / 3600, 1)
    group['thread_lifetime_h'] = round((group['time'].max()-group['time'].min()).total_seconds() / 3600, 1)
    group['number_of_comments'] = len(group)
    group['number_of_users'] = len(group[user_id].unique())

    percentiles = add_percentile_column(group)['percentile']

    user_counts = group.groupby(user)[user].count()
    group['number_of_comments_by_user_in_thread'] = group[user].map(user_counts)
    group['user_appearances'] = group.groupby(user).cumcount() + 1

    data.loc[group.index, 'percentile'] = percentiles
    data.loc[group.index,'sequential_number_of_comment_by_user_in_thread'] = group['user_appearances']
    data.loc[group.index, 'number_of_comments_by_user_in_thread'] = group['number_of_comments_by_user_in_thread']
    data.loc[group.index, 'thread_birth'] = group['thread_birth']
    data.loc[group.index,'temporal_distance_birth_h'] = group['temporal_distance_birth_h']
    data.loc[group.index, 'thread_lifetime_h'] = group['thread_lifetime_h']
    data.loc[group.index, 'number_of_users'] = group['number_of_users']
    data.loc[group.index, 'number_of_comments'] = group['number_of_comments']
    progress_bar.update(1)

progress_bar.close()

data = data.sort_values(by=[user, created_at])
data[created_at] = pd.to_datetime(data[created_at],format='ISO8601')
data['temporal_distance_from_previous_comment_h'] = data.groupby(user)[created_at].diff().dt.total_seconds() / 3600
data = data.sort_values(by=[user, created_at])
data['IAT_user'] = data.groupby(user)[created_at].diff().dt.total_seconds()
data = data.sort_values(by=[user,post_id, created_at])
data['IAT_user_thread'] = data.groupby([user,post_id])[created_at].diff().dt.total_seconds()
data = data.sort_values(by=[post_id,created_at])
data['IAT_thread'] = data.groupby(post_id)[created_at].diff().dt.total_seconds()

data=data[[comment_id,user,post_id,  toxicity_score,
       created_at, 'percentile',
       'sequential_number_of_comment_by_user_in_thread',
       'number_of_comments_by_user_in_thread', 'thread_birth',
       'temporal_distance_birth_h', 'thread_lifetime_h', 'number_of_users', 'number_of_comments', 'temporal_distance_from_previous_comment_h',
       'IAT_user', 'IAT_user_thread', 'IAT_thread',topic,social]]
data.columns=['comment_id', 'user_id', 'post_id', 'toxicity_score', 'created_at',
       'percentile', 'sequential_number_of_comment_by_user_in_thread',
       'number_of_comments_by_user_in_thread', 'thread_birth',
       'temporal_distance_birth_h', 'thread_lifetime_h', 'number_of_users',
       'number_of_comments', 'temporal_distance_from_previous_comment_h',
       'IAT_user', 'IAT_user_thread', 'IAT_thread', 'topic', 'social']

Add variables...: 100%|██████████| 1396/1396 [00:22<00:00, 61.19it/s]


In [94]:
data.to_csv('/kaggle/working/'+platform+'_processed.csv', index=False)